In [346]:
import pandas as pd
import json

In [347]:
#Запсиываем json в dataframe
with open("trial_task.json") as data:
    df = pd.read_json(data)


In [348]:
df

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."
...,...,...,...,...
95,28106,хутор близ Диканьки,-30,"[{'product': 'подписка на suppi-блог', 'price'..."
96,98100,остров невезения,-10,"[{'product': 'автограф Стаса Барецкого', 'pric..."
97,79293,отель Лето,-75,"[{'product': 'статуэтка Ленина', 'price': 200,..."
98,2930,Мордор,-30,"[{'product': 'плюмбус', 'price': 250, 'quantit..."


In [349]:
# Задание 1 - найти тариф

# Разворачиваем json продуктов
exploded_df = df.explode('products', ignore_index=True)
pd.json_normalize(exploded_df['products'])
# Получаем таблицу с продуктами совмещённую с основным датфреймом
exploded_df = pd.concat([exploded_df.drop('products', axis = 1), pd.json_normalize(exploded_df['products'])], axis = 1)
# Словарь для будущей конвертации в датафрейм
data = {
    'order_id' : [],
    'delivery_rate': []
}
# Перебираем все заказы
for ord_id in df['order_id']:
    # Если айди заказа находится в таблице продутов получаем сумму продуктов в этом заказе
    sum = exploded_df.loc[exploded_df['order_id'] == ord_id]['quantity'].sum()
    # Заполняем словарь
    data['delivery_rate'].append(sum)
    data['order_id'].append(ord_id)
# Сливаем словари
delivery_rate = pd.DataFrame(data)
df = df.merge(delivery_rate)
# Перезаписываем тариф в соответсвии с формулой
df['delivery_rate'] = (df['highway_cost'] /  df['delivery_rate']) * -1
# Вывод
df

,order_id,warehouse_name,highway_cost,products,delivery_rate
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':...",10.0
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,...",15.0
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,...",25.0
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,...",10.0
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric...",25.0
...,...,...,...,...,...
95,28106,хутор близ Диканьки,-30,"[{'product': 'подписка на suppi-блог', 'price'...",15.0
96,98100,остров невезения,-10,"[{'product': 'автограф Стаса Барецкого', 'pric...",5.0
97,79293,отель Лето,-75,"[{'product': 'статуэтка Ленина', 'price': 200,...",25.0
98,2930,Мордор,-30,"[{'product': 'плюмбус', 'price': 250, 'quantit...",10.0


In [350]:
# Задание 2 - Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')
# Развернули продукты, без слияния с основной таблицей
exploded_df = df.explode('products', ignore_index=True)
prod_df = pd.json_normalize(exploded_df['products'])

# Группируем по продукту
g = prod_df.groupby('product',sort=False)
out = pd.DataFrame()
# Количество = сумма кол-ва всех продуктов
out['quantity']=g['quantity'].sum()
# Доход = сумма цен всех продуктов
out['income']=g['price'].sum()
# Вытаскиваем затраты на каждый продукт из основной таблицы
highway_cost_df = pd.concat([exploded_df.drop('products', axis = 1), pd.json_normalize(exploded_df['products'])], axis = 1)
# Группируем по продуктам
highway_cost_df_g = highway_cost_df.groupby('product',sort=False)
# Сумма затрат на каждый продукт
out['expenses'] = highway_cost_df_g["highway_cost"].sum()
# Прибыль = Доходы - Расходы
out['profit'] = out['income'] + out['expenses']
out

,quantity,income,expenses,profit
product,,,,
ломтик июльского неба,42,9450,-1210,8240
билет в Израиль,58,29000,-2305,26695
статуэтка Ленина,68,7400,-2235,5165
зеленая пластинка,61,310,-2260,-1950
автограф Стаса Барецкого,48,16200,-2090,14110
плюмбус,65,8250,-2255,5995
подписка на suppi-блог,33,2850,-1415,1435


In [351]:
# Задание 3 - Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

# Как и раньше раскрываем таблицу
exploded_df = df.explode('products', ignore_index=True)
pd.json_normalize(exploded_df['products'])
exploded_df = pd.concat([exploded_df.drop('products', axis = 1), pd.json_normalize(exploded_df['products'])], axis = 1)
# Доход = цена * кол-во
exploded_df["profit"] = exploded_df['price'] * exploded_df['quantity']
# Сортировка по заказам
g = exploded_df.groupby('order_id', sort=False)
order_profit = pd.DataFrame()
# Прибыль = доход - затраты
order_profit['order_profit'] =  g["profit"].sum() + g["highway_cost"].mean()
# Вывод средней прибыли
print(order_profit['order_profit'].mean())
# Вывод таблицы
order_profit

1352.3


,order_profit
order_id,
11973,3980.0
62239,985.0
85794,-30.0
33684,1980.0
25824,975.0
...,...
28106,320.0
98100,1590.0
79293,1175.0


In [352]:
# Задание 4 - Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)
# Открываем таблицу
exploded_df = df.explode('products', ignore_index=True)
pd.json_normalize(exploded_df['products'])
exploded_df = pd.concat([exploded_df.drop('products', axis = 1), pd.json_normalize(exploded_df['products'])], axis = 1)
# Расчёт прибыли для каждого отдельного продукта за вычетом расходов
exploded_df['profit'] = (exploded_df['price'] - exploded_df['delivery_rate']) * exploded_df['quantity']
# Прибыль для каждого склада
g =  exploded_df.groupby('warehouse_name')['profit'].sum()
# Добавляем в таблицу прибыль склада для дальнейших расчётов
exploded_df = pd.merge(exploded_df, g, how='left', left_on='warehouse_name', right_on='warehouse_name', suffixes=("_product", "_warehouse"))
# Процент прибыли продукта = (Прибыль от продажи продукта / Общая прибыль склада)  * 100
exploded_df['percent_profit_product_of_warehouse'] = exploded_df['profit_product'] / exploded_df['profit_warehouse']  * 100
# Убираем временные столбцы для расчётов
exploded_df = exploded_df.drop(["order_id", "price", "highway_cost", "profit_product", "profit_warehouse"], axis=1)

exploded_df

,warehouse_name,delivery_rate,product,quantity,percent_profit_product_of_warehouse
0,Мордор,10.0,ломтик июльского неба,1,2.580645
1,Мордор,10.0,билет в Израиль,3,17.419355
2,Мордор,10.0,статуэтка Ленина,3,3.343109
3,хутор близ Диканьки,15.0,билет в Израиль,1,2.645005
4,отель Лето,25.0,зеленая пластинка,2,-0.110906
...,...,...,...,...,...
192,отель Лето,25.0,автограф Стаса Барецкого,1,2.125693
193,отель Лето,25.0,ломтик июльского неба,1,1.571165
194,Мордор,10.0,плюмбус,2,2.815249
195,Мордор,10.0,плюмбус,1,1.407625


In [353]:
# Сортируем по убыванию
exploded_df.sort_values(by=['percent_profit_product_of_warehouse'], ascending=False)
# Накопленный процент
exploded_df['accumulated_percent_profit_product_of_warehouse'] = exploded_df['percent_profit_product_of_warehouse'].cumsum()
exploded_df

,warehouse_name,delivery_rate,product,quantity,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,Мордор,10.0,ломтик июльского неба,1,2.580645,2.580645
1,Мордор,10.0,билет в Израиль,3,17.419355,20.000000
2,Мордор,10.0,статуэтка Ленина,3,3.343109,23.343109
3,хутор близ Диканьки,15.0,билет в Израиль,1,2.645005,25.988114
4,отель Лето,25.0,зеленая пластинка,2,-0.110906,25.877208
...,...,...,...,...,...,...
192,отель Лето,25.0,автограф Стаса Барецкого,1,2.125693,492.312836
193,отель Лето,25.0,ломтик июльского неба,1,1.571165,493.884000
194,Мордор,10.0,плюмбус,2,2.815249,496.699250
195,Мордор,10.0,плюмбус,1,1.407625,498.106874


In [354]:
# Функция-фильтр
def assign_category(value):
    if value <= 70:
        return 'A'
    elif value <= 90:
        return 'B'
    else:
        return 'C'
# Добавляем новый столбец через фильтр
exploded_df['category'] = exploded_df['accumulated_percent_profit_product_of_warehouse'].apply(assign_category)
exploded_df

,warehouse_name,delivery_rate,product,quantity,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,Мордор,10.0,ломтик июльского неба,1,2.580645,2.580645,A
1,Мордор,10.0,билет в Израиль,3,17.419355,20.000000,A
2,Мордор,10.0,статуэтка Ленина,3,3.343109,23.343109,A
3,хутор близ Диканьки,15.0,билет в Израиль,1,2.645005,25.988114,A
4,отель Лето,25.0,зеленая пластинка,2,-0.110906,25.877208,A
...,...,...,...,...,...,...,...
192,отель Лето,25.0,автограф Стаса Барецкого,1,2.125693,492.312836,C
193,отель Лето,25.0,ломтик июльского неба,1,1.571165,493.884000,C
194,Мордор,10.0,плюмбус,2,2.815249,496.699250,C
195,Мордор,10.0,плюмбус,1,1.407625,498.106874,C
